In [27]:
import os
from PIL import Image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# Split set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Path to image folders, 
data_path = fr'../Datasets/train'
# os.listdir(data_path)

def remove_white_background(pixels):
    for pixel in pixels:
        pixel = list(pixel)
        if ((256 > pixel[0] > 200) and (256 > pixel[1] > 200) and (256 > pixel[2] > 200)):
            pixel[0] = 0
            pixel[1] = 0
            pixel[2] = 0

    return pixels


def redify(pixels):
    return [r for r, g, b in pixels]

                
def greenify(pixels):
    return [g for r, g, b in pixels]



def blueify(pixels):
    return [b for r, g, b in pixels]


def get_rgb_pixels_onehut_labels(src):
    print("Starting...")
    newPixels = []
    y = np.empty(shape=[0, 1])

    for subdir in os.listdir(src):
        current_path = os.path.join(src, subdir)
        for file in os.listdir(current_path):
            img = Image.open(os.path.join(current_path, file))
            imgResize = img.resize((24,24))
            pixels = list(imgResize.getdata())
            pixels = remove_white_background(pixels)
            newPixels.append(pixels)
            y = np.append(y, subdir)
    print(LabelBinarizer().fit_transform(y))
    return newPixels, LabelBinarizer().fit_transform(y) # OneHot encode y

def process_files(src):
    X_red_train = []
    X_red_validation = []
    X_red_test = []
    X_green_train = []
    X_green_validation = []
    X_green_test = []
    X_blue_train = []
    X_blue_validation = []
    X_blue_test = []
    all_pixels, y = get_rgb_pixels_onehut_labels(src)
        
    X_train, X_validation, y_train, y_validation = train_test_split(all_pixels, y, test_size=0.20, random_state=1)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
    
    for pixels in X_train:       
        X_red_train.append(redify(pixels.copy()))
        X_green_train.append(greenify(pixels.copy()))
        X_blue_train.append(blueify(pixels.copy()))
        
    for pixels in X_validation:       
        X_red_validation.append(redify(pixels.copy()))
        X_green_validation.append(greenify(pixels.copy()))
        X_blue_validation.append(blueify(pixels.copy()))
        
    for pixels in X_test:       
        X_red_test.append(redify(pixels.copy()))
        X_green_test.append(greenify(pixels.copy()))
        X_blue_test.append(blueify(pixels.copy()))
    
    
    print("Finished")
    return np.asarray(X_red_train), np.asarray(X_red_validation), np.asarray(X_red_test), np.asarray(X_green_train), np.asarray(X_green_validation), np.asarray(X_green_test), np.asarray(X_blue_train), np.asarray(X_blue_validation), np.asarray(X_blue_test), y_train, y_validation, y_test

In [28]:
# Call process_files and assign variables
X_red_train, X_red_validation, X_red_test, X_green_train, X_green_validation, X_green_test, X_blue_train, X_blue_validation, X_blue_test, y_train, y_validation, y_test
# Fit/train train-datasets and store prediction vectors in variables
from sklearn import svm
from sklearn.neural_network import MLPClassifier

# Red
mlpc_red =  MLPClassifier(hidden_layer_sizes=(576), activation='relu', solver='adam')
mlpc_red.fit(X_red_train, y_train)
vector_red = mlpc_red.predict(X_red_train)

# Green
mlpc_green = MLPClassifier(hidden_layer_sizes=(576), activation='relu', solver='adam')
mlpc_green.fit(X_green_train, y_train)
vector_green = mlpc_green.predict(X_green_train)

# Blue
mlpc_blue = MLPClassifier(hidden_layer_sizes=(576), activation='relu', solver='adam')
mlpc_blue.fit(X_blue_train, y_train)
vector_blue = mlpc_blue.predict(X_blue_train)

In [29]:
from sklearn.metrics import classification_report,confusion_matrix
#print(confusion_matrix(mlpc_green.predict(X_green_test), y_test))
print("Red model")
print(classification_report(mlpc_red.predict(X_red_validation), y_validation))
print("Green model")
print(classification_report(mlpc_green.predict(X_green_validation), y_validation))
print("Blue model")
print(classification_report(mlpc_blue.predict(X_blue_validation), y_validation))



Red model
              precision    recall  f1-score   support

           0       1.00      0.90      0.95        83
           1       1.00      0.97      0.98        99
           2       1.00      1.00      1.00       111
           3       1.00      0.98      0.99        65
           4       1.00      1.00      1.00        86
           5       0.99      1.00      1.00       144

   micro avg       1.00      0.98      0.99       588
   macro avg       1.00      0.98      0.99       588
weighted avg       1.00      0.98      0.99       588
 samples avg       1.00      0.99      0.99       588

Green model
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        75
           1       1.00      1.00      1.00        96
           2       1.00      1.00      1.00       111
           3       1.00      1.00      1.00        64
           4       1.00      1.00      1.00        86
           5       1.00      1.00      1.00       145

 

c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(composite_vector_train_results.shape)
print(composite_vector_train.shape)

composite_vector_train_results = np.stack((vector_red, vector_green, vector_blue)).reshape(3,-1)
composite_vector_train = np.stack((X_red_train, X_green_train, X_blue_train)).reshape(3,-1)

mlpc_comp = MLPClassifier()
mlpc_comp.fit(composite_vector_train, composite_vector_train_results)
print("Finished")